In [1]:
!apt-get update -qq
!apt-get install -y build-essential python3-dev default-jdk libmecab-dev mecab-ipadic-utf8
!pip install --upgrade pip setuptools wheel


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
default-jdk is already the newest version (2:1.11-72build2).
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 61.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [konlpy]


In [3]:
# ====== 1) 데이터 ======
import pandas as pd

data = [
    {"title":"기생충", "genres":"드라마 스릴러", "overview":"가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러"},
    {"title":"옥자", "genres":"드라마 모험", "overview":"거대한 동물 옥자와 소녀의 우정을 지키기 위한 모험"},
    {"title":"부산행", "genres":"액션 스릴러 좀비", "overview":"좀비가 창궐한 열차에서 벌어지는 생존기"},
    {"title":"더 웨일", "genres":"드라마", "overview":"세상과 단절된 채 살던 교수가 딸과의 관계를 회복하려 한다"},
    {"title":"인터스텔라", "genres":"SF 드라마 모험", "overview":"인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정"},
    {"title":"그래비티", "genres":"SF 스릴러", "overview":"우주에서 고립된 우주비행사들의 생존기"},
    {"title":"매드맥스 분노의 도로", "genres":"액션 모험", "overview":"사막의 종말 세계에서 카체이스와 해방을 향한 싸움"},
    {"title":"인셉션", "genres":"SF 스릴러 액션", "overview":"타인의 꿈 속에 들어가 아이디어를 주입하는 팀의 작전"},
    {"title":"라라랜드", "genres":"드라마 로맨스 음악", "overview":"배우 지망생과 재즈 피아니스트의 사랑과 꿈"},
    {"title":"위플래쉬", "genres":"드라마 음악", "overview":"천재 드러머 지망생과 혹독한 스승의 집착과 성장"},
    {"title":"코코", "genres":"애니메이션 가족 음악", "overview":"음악을 사랑하는 소년이 죽은 자의 나라에서 가족의 비밀을 찾는다"},
    {"title":"토이 스토리", "genres":"애니메이션 가족 모험", "overview":"장난감들의 우정과 모험이 펼쳐지는 따뜻한 이야기"},
    {"title":"가디언즈 오브 갤럭시", "genres":"SF 액션 코미디", "overview":"우주 악당들로부터 은하계를 지키는 유쾌한 히어로 팀"},
    {"title":"닥터 스트레인지", "genres":"SF 액션 판타지", "overview":"교만한 천재 외과의가 마법사가 되어 세상을 위협으로부터 지킨다"},
    {"title":"셰이프 오브 워터", "genres":"드라마 판타지 로맨스", "overview":"말 못 하는 여성과 수중 생명체 사이의 특별한 사랑"}
]
df = pd.DataFrame(data)

# ====== 2) 형태소 분석기 준비 (Mecab 우선, 실패 시 Okt) ======
from konlpy.tag import Okt
tokenizer_name = "okt"
okt = Okt()
mecab = None

try:
    # konlpy의 Mecab은 시스템에 mecab-ko 사전이 있어야 함.
    from konlpy.tag import Mecab
    mecab = Mecab()
    tokenizer_name = "mecab"
except Exception as e:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")

print("사용 토크나이저:", tokenizer_name)

# ====== 3) 불용어 정의 ======
# 프로젝트에 맞게 확장/수정하세요.
stopwords = set("""
은 는 이 가 을 를 에 에서 으로 도 와 과 의 로 으로부터 부터 보다 뿐만 아니라 그리고 그러나 하지만 또한
하다 되다 이다 있다 없다 모든 같은 매우 너무 가장 대한 위해 위협 으며 으면서 등 등등 각 각종 해당
저 그 이 그녀 그것 이것 저것 그들 우리 너희 여러분 또한 또한의 등에서 등의 등으로 또 또는
""".split())

# 자주 등장하지만 정보량이 낮은 일반 토큰 + 장르/상투어 일부 추가
stopwords.update("""
드라마 스릴러 액션 SF 모험 코미디 로맨스 판타지 애니메이션 가족 음악
영화 이야기 작품 장면 여정 모험 팀 히어로 악당 생존기 작전 관계 사랑 꿈
""".split())

# ====== 4) 전처리 함수 (형태소 분석 + 불용어 제거) ======
def tokenize_ko(text: str):
    if not isinstance(text, str):
        return []
    text = text.strip()
    if tokenizer_name == "mecab":
        tokens = mecab.morphs(text)
    else:
        tokens = okt.morphs(text, stem=True)  # Okt는 표제어화 옵션(stem=True) 유용
    # 한글/영문/숫자만 남기고, 길이 1 토큰/불용어 제거
    clean = []
    for t in tokens:
        t = t.lower()
        if t in stopwords:
            continue
        if len(t) < 2:  # 1글자 토큰 제거(원하면 완화 가능)
            continue
        clean.append(t)
    return clean

# ====== 5) 텍스트 결합 → 토큰화 → 전처리 ======
df["raw_text"] = (df["genres"].fillna("") + " " + df["overview"].fillna(""))
df["tokens"] = df["raw_text"].map(tokenize_ko)
# TF-IDF는 기본적으로 문자열을 받으니, 공백으로 재결합
df["text_for_tfidf"] = df["tokens"].map(lambda toks: " ".join(toks))

# ====== 6) TF-IDF + 코사인 유사도 ======
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 이미 토큰화/전처리된 문자열이므로 token_pattern을 공백 기준으로만 보게 설정
vectorizer = TfidfVectorizer(token_pattern=r"[^ ]+", ngram_range=(1,2), max_features=8000)
tfidf = vectorizer.fit_transform(df["text_for_tfidf"])
cosine_sim = cosine_similarity(tfidf, tfidf)

# ====== 7) 추천 함수 ======
from difflib import get_close_matches

title_to_idx = {t.lower(): i for i, t in enumerate(df["title"])}

def _resolve_title(user_title: str):
    key = user_title.strip().lower()
    if key in title_to_idx:
        return key
    cand = get_close_matches(key, list(title_to_idx.keys()), n=1, cutoff=0.5)
    return cand[0] if cand else None

def recommend(title: str, top_n: int = 5, include_score: bool = True):
    key = _resolve_title(title)
    if not key:
        print(f"[경고] '{title}' 제목을 찾지 못했습니다. 데이터셋 제목 중 하나를 입력하세요.")
        return pd.DataFrame()
    idx = title_to_idx[key]
    sims = list(enumerate(cosine_sim[idx]))
    sims = sorted([(i,s) for i,s in sims if i != idx], key=lambda x: -x[1])[:top_n]
    rec_idx = [i for i,_ in sims]
    out = df.loc[rec_idx, ["title","genres","overview"]].copy()
    if include_score:
        out["similarity"] = [round(s, 4) for _, s in sims]
    return out.reset_index(drop=True)

# ====== 8) 사용 예시 ======
print("=== [인셉션] 비슷한 영화 ===")
display(recommend("인셉션", top_n=5))

print("=== [라라랜드] 비슷한 영화 ===")
display(recommend("라라랜드", top_n=5))

print("=== [기생충] 비슷한 영화 (근사매칭 허용) ===")
display(recommend("기생충", top_n=5))

사용 토크나이저: mecab
=== [인셉션] 비슷한 영화 ===


,title,genres,overview,similarity
0,가디언즈 오브 갤럭시,SF 액션 코미디,우주 악당들로부터 은하계를 지키는 유쾌한 히어로 팀,0.0462
1,그래비티,SF 스릴러,우주에서 고립된 우주비행사들의 생존기,0.0444
2,닥터 스트레인지,SF 액션 판타지,교만한 천재 외과의가 마법사가 되어 세상을 위협으로부터 지킨다,0.0413
3,인터스텔라,SF 드라마 모험,인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정,0.0412
4,기생충,드라마 스릴러,가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러,0.0000


=== [라라랜드] 비슷한 영화 ===


,title,genres,overview,similarity
0,위플래쉬,드라마 음악,천재 드러머 지망생과 혹독한 스승의 집착과 성장,0.082
1,기생충,드라마 스릴러,가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러,0.000
2,옥자,드라마 모험,거대한 동물 옥자와 소녀의 우정을 지키기 위한 모험,0.000
3,부산행,액션 스릴러 좀비,좀비가 창궐한 열차에서 벌어지는 생존기,0.000
4,더 웨일,드라마,세상과 단절된 채 살던 교수가 딸과의 관계를 회복하려 한다,0.000


=== [기생충] 비슷한 영화 (근사매칭 허용) ===


,title,genres,overview,similarity
0,부산행,액션 스릴러 좀비,좀비가 창궐한 열차에서 벌어지는 생존기,0.0721
1,옥자,드라마 모험,거대한 동물 옥자와 소녀의 우정을 지키기 위한 모험,0.0000
2,더 웨일,드라마,세상과 단절된 채 살던 교수가 딸과의 관계를 회복하려 한다,0.0000
3,인터스텔라,SF 드라마 모험,인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정,0.0000
4,그래비티,SF 스릴러,우주에서 고립된 우주비행사들의 생존기,0.0000
